### E02
Split up the dataset randomly into 80% train set, 10% dev set, 10% test set. 

Train the bigram and trigram models only on the training set. 

Evaluate them on dev and test splits. What can you see?

### Setting up datasets
- Load all the names into the list
- Split the list into 80% train set, 10% dev set, 10% test set
- Form stoi and itos dictionaries

In [78]:
# Load the dataset into a list
with open('names.txt', 'r') as file:
    names = file.read().split()

print('Total names:', len(names))
print(min(len(item) for item in names), max(len(item) for item in names))

Total names: 32033
2 15


In [79]:
# Splitting names into 80% train set, 10% dev set, 10% test set
import random

random.shuffle(names) # Shuffles the list in-place

split_1, split_2 = int(0.8*len(names)), int(0.9*len(names))
train_list = names[:split_1]
dev_list = names[split_1:split_2]
test_list = names[split_2:]

In [80]:
print(f'Training set: {len(train_list)/len(names)*100.:4f} %')
print(f'Dev set: {len(dev_list)/len(names)*100.:4f} %')
print(f'Test set: {len(test_list)/len(names)*100.:4f} %')

Training set: 79.998751 %
Dev set: 9.999063 %
Test set: 10.002185 %


In [81]:
# Form stoi and itos
vocab = sorted(list(set(''.join(names))))
stoi = {s: i+1 for i, s in enumerate(vocab)}
stoi['.'] = 0
itos = {stoi[s]: s for s in stoi}

### Bigram: training
- Perform Gradient descent and update weights only based on training list

In [131]:
import torch

# Preparing bigrams
xs, ys = [], []
for name in train_list:
    name = ['.'] + list(name) + ['.']
    for xi, yi in zip(name, name[1:]):
        xs.append(stoi[xi])
        ys.append(stoi[yi])

xs = torch.tensor(xs)
ys = torch.tensor(ys)

# Encoding the inputs
import torch.nn.functional as F
xenc = F.one_hot(xs, num_classes=27).float()
yenc = F.one_hot(ys, num_classes=27).float()

In [132]:
# Initialization
g = torch.Generator().manual_seed(2147483647)
W_bigram = torch.randn(27, 27, generator=g, requires_grad=True)
logits = (xenc @ W_bigram)
counts = logits.exp()
probs = counts / counts.sum(1, keepdims=True)

In [111]:
num = ys.nelement() # number of examples

# Gradient descent
for _ in range(100):
    # Forward pass
    logits = xenc @ W_bigram # predicted log counts

    # softmax (next two lines)
    counts = logits.exp() # predicted counts
    probs = counts / counts.sum(1, keepdims=True) # calculating probablities
    
    loss = -probs[torch.arange(num), ys].log().mean()

    print(loss.item())

    # Backward pass
    W_bigram.grad = None
    loss.backward()

    # Update the weights
    W_bigram.data += -50 * W_bigram.grad

3.759140729904175
3.3714535236358643
3.1542441844940186
3.0203638076782227
2.9275906085968018
2.8602118492126465
2.809485673904419
2.7698166370391846
2.7377521991729736
2.7111473083496094
2.688631296157837
2.6692991256713867
2.6525282859802246
2.6378684043884277
2.6249759197235107
2.613579273223877
2.603454351425171
2.5944154262542725
2.5863077640533447
2.579000473022461
2.572385311126709
2.566370725631714
2.560879945755005
2.5558481216430664
2.551220417022705
2.546949625015259
2.542996883392334
2.539328098297119
2.535914182662964
2.5327305793762207
2.529755115509033
2.5269699096679688
2.524357557296753
2.521904230117798
2.519596815109253
2.517423391342163
2.515374183654785
2.513439416885376
2.511610507965088
2.5098795890808105
2.50823974609375
2.5066845417022705
2.5052077770233154
2.5038044452667236
2.5024688243865967
2.50119686126709
2.4999845027923584
2.4988279342651367
2.497722864151001
2.496666669845581
2.4956560134887695
2.4946885108947754
2.4937610626220703
2.4928712844848633
2.

### Bigram: loss on dev and test sets
- Prepare xs & corresponding ys dataset on dev & test
- Evaluate loss of Dev set and test set

In [104]:
xs_dev, xs_test, ys_dev, ys_test = [], [], [], []
for name in dev_list:
    name = ['.'] + list(name) + ['.']
    for ch1, ch2 in zip(name, name[1:]):
        xs_dev.append(stoi[ch1])
        ys_dev.append(stoi[ch2])

for name in test_list:
    name = ['.'] + list(name) + ['.']
    for ch1, ch2 in zip(name, name[1:]):
        xs_test.append(stoi[ch1])
        ys_test.append(stoi[ch2])

xs_dev = torch.tensor(xs_dev)
ys_dev = torch.tensor(ys_dev)
xs_test = torch.tensor(xs_test)
ys_test = torch.tensor(ys_test)

num_dev = len(xs_dev)
num_test = len(xs_test)

In [105]:
probs.shape

torch.Size([182521, 27])

In [106]:
dev_loss = -probs[torch.arange(num_dev), ys_dev].log().mean()
test_loss = -probs[torch.arange(num_test), ys_test].log().mean()

In [134]:
print(dev_loss.item(), test_loss.item())

3.31862473487854 3.3068392276763916


### Trigrams: training

In [114]:
xs_train, ys_train = [], []
for name in train_list:
    name = ['.'] + list(name) + ['.']
    for ch1, ch2, ch3 in zip(name, name[1:], name[2:]):
        xs_train.append([stoi[ch1], stoi[ch2]])
        ys_train.append(stoi[ch3])

xs_train = torch.tensor(xs_train)
ys_train = torch.tensor(ys_train)

In [ ]:
# One hot encoding of the inputs
import torch.nn.functional as F
xenc_train = F.one_hot(xs_train, num_classes=27).float() # casting to float32 to match dtype of Weight
xenc_train = xenc_train.view(xenc_train.shape[0], 2*27) 
yenc_train = F.one_hot(ys_train, num_classes=27).float()

num_train = ys_train.nelement()

In [119]:
g = torch.Generator().manual_seed(2147483647)
W_tri = torch.randn(27*2, 27, generator=g, requires_grad=True)

In [126]:
for _ in range(10):
    # Forward pass
    logits = (xenc_train @ W_tri)
    counts = logits.exp()
    # Normalize the count to get probability distribution
    probs = counts / counts.sum(1, keepdims=True)

    # print(torch.arange(num))
    # print(ys.shape)
    # print(logits.shape, probs.shape)
    loss = -probs[torch.arange(num_train), ys_train].log().mean()
    print(loss.item())
    
    # Backward pass
    W_tri.grad = None
    loss.backward()
    
    # Update the weights
    W_tri.data += -10 * W_tri.grad

2.5036652088165283
2.4992551803588867
2.4950172901153564
2.490922451019287
2.486955404281616
2.483107805252075
2.4793732166290283
2.4757466316223145
2.4722235202789307
2.468799114227295


### Trigram: loss on dev and test sets

In [122]:
xs_dev_tri, xs_test_tri, ys_dev_tri, ys_test_tri = [], [], [], []
for name in dev_list:
    name = ['.'] + list(name) + ['.']
    for ch1, ch2, ch3 in zip(name, name[1:], name[2:]):
        xs_dev_tri.append([stoi[ch1], stoi[ch2]])
        ys_dev_tri.append(stoi[ch3])

for name in test_list:
    name = ['.'] + list(name) + ['.']
    for ch1, ch2, ch3 in zip(name, name[1:], name[2:]):
        xs_test_tri.append([stoi[ch1], stoi[ch2]])
        ys_test_tri.append(stoi[ch3])

xs_dev_tri = torch.tensor(xs_dev_tri)
ys_dev_tri = torch.tensor(ys_dev_tri)
xs_test_tri = torch.tensor(xs_test_tri)
ys_test_tri = torch.tensor(ys_test_tri)

num_dev_tri = len(ys_dev_tri)
num_test_tri = len(ys_test_tri)

In [127]:
dev_loss_tri = -probs[torch.arange(num_dev_tri), ys_dev_tri].log().mean()
test_loss_tri = -probs[torch.arange(num_test_tri), ys_test_tri].log().mean()

In [128]:
print('Dev loss on trigrams:', dev_loss_tri.item())
print('Test loss on trigrams:', test_loss_tri.item())

Dev loss on trigrams: 3.237778425216675
Test loss on trigrams: 3.236469030380249
